<img src="images/dask_horizontal.svg"
     width="45%"
     alt="Dask logo\">
     
# Dask Delayed

This notebook covers Dask's `delayed` interface and how it can be used to parallelize existing Python code and custom algorithms. Let's start by looking at a basic, non-parallelized example and then see how `dask.delayed` can help.

# Basic example

In the code cell below, we have two Python functions, `inc` and `add`, which increment and add together their inputs, respectively. We call these functions on input values to produce some output which we then print.

*Tip*: the `%%time` at the top of the cell tells Jupyter to print out how long it took the cell to run.

In [ ]:
%%time

import time

def inc(i):
    time.sleep(1)
    return i + 1

def add(a, b):
    time.sleep(1)
    return a + b

a = 1
b = 12

# Increment a and b
c = inc(a)
d = inc(b)

# Add results together
output = add(c, d)

print(f"{output = }")

The steps in this computation can be encoded in the following task graph shown below

![](images/inc-add.svg)

In the above task graph:

1. Circular nodes in the graph are Python function calls

2. Square nodes are Python objects that are created by one task as output and can be used as inputs in another task

3. Arrows represent dependencies between tasks

From looking at the task graph, we can see there's an opportunity for parallelism here! The two `inc` calls are totally independent of one another, so they could be run at the same time in parallel. Let's see how we can use Dask's `delayed` interface to do this.

# `dask.delayed` decorator

Dask's `delayed` interface consists of a single `delayed` decorator which allows you to build up complex task graphs by lightly annotating normal Python functions. Dask can then execute the task graph (potentially in parallel). The idea is that you can take your existing Python code, apply a few `delayed` decorators, and then have a parallel version of your code.

Let's revist our `inc` / `add` example from before:

In [ ]:
%%time

import time
from dask import delayed    # Import the delayed decorator

@delayed                    # Wrap inc with delayed
def inc(i):
    time.sleep(1)
    return i + 1

@delayed                    # Wrap add with delayed
def add(a, b):
    time.sleep(1)
    return a + b

a = 1
b = 12

# Increment a and b
c = inc(a)
d = inc(b)

# Add results together
output = add(c, d)

print(f"{output = }")

That happened much faster! But notice that the above cell didn't print the expected result of `15`, instead it printed a `Delayed` object.

That's because Dask `delayed` works by wraping function calls and **delaying their execution** (hence the name "delayed"). Instead of returning the result of a function call, `delayed` functions return `Delayed` objects which keep track of what we want to compute by automatically building a task graph for us.

You can see the task graph for a `Delayed` object by calling its `visualize` method:

In [ ]:
output.visualize()

To actually compute a result of a `Delayed` object, call its `compute` method which will tell Dask to compute the task graph in parallel.

In [ ]:
%%time

# Compute result
result = output.compute()
print(f"{result = }")

Notice that the parallel version of this computation took ~2s while the non-parallel version took ~3s. Why do you think that is?

``Delayed`` objects support several standard Python operations, each of which creates another ``Delayed`` object representing the result:

- Arithamtic operators, e.g. `*`, `-`, `+`
- Item access and slicing, e.g. `x[0]`, `x[1:3]`
- Attribute access, e.g. `x.size`
- Method calls, e.g. `x.index(0)`

Using `delayed` functions, we can easily build up a task graph for the particular computation we want to perform.

In [ ]:
result = inc(5)
result.visualize()

In [ ]:
result.compute()

In [ ]:
result = inc(5) * inc(7)
result.visualize()

In [ ]:
result.compute()

In [ ]:
result = (inc(5) * inc(7)) + 2
result.visualize()

In [ ]:
result.compute()

# Exercise 1: Parallelize a for-loop

Below we define three functions: `inc`, `double`, and `add`. We use these functions to perform some operations on a list (assigned to the `data` variable). For this exercise, use `delayed` to run these operations in parallel.

In [ ]:
%%time

import time

def inc(x):
    time.sleep(0.5)
    return x + 1

def double(x):
    time.sleep(0.5)
    return 2 * x

def add(x, y):
    time.sleep(0.5)
    return x + y

data = list(range(10))

output = []
for x in data:
    a = inc(x)
    b = double(x)
    c = add(a, b)
    output.append(c)

total = sum(output)
total

In [ ]:
# Your solution here

In [ ]:
%load solutions/delayed-1.py

# Exercise 2: Parallelize a for-loop with conditional flow

This exercise is similar to the previous, but now instead of always having `a = inc(x)` we sometimes increment `x` and sometimes double `x` depending on if `x` is an even number or not. For this exercise, we again want to use `delayed` to run these operations in parallel.

In [ ]:
import time

def inc(x):
    time.sleep(0.5)
    return x + 1

def double(x):
    time.sleep(0.5)
    return 2 * x

def add(x, y):
    time.sleep(0.5)
    return x + y

def is_even(x):
    return not x % 2

In [ ]:
%%time

data = list(range(10))

output = []
for x in data:
    if is_even(x):
        a = inc(x)
    else:
        a = double(x)
    b = double(x)
    c = add(a, b)
    output.append(c)

total = sum(output)
total

In [ ]:
# Your solution here

In [ ]:
%load solutions/delayed-2.py

# Exercise 3: Parallelize Pandas' `read_csv`

For this exercise we'll use CSV files from NYC's flight dataset. You can download the CSV files by running the cell below.

In [ ]:
# Run this cell to download NYC flight dataset
%run prep.py -d flights

We can then use Python's `glob` module to get a list of all the CSV files in the dataset:

In [ ]:
import glob

files = glob.glob("data/nycflights/*.csv")
files

[Pandas' `read_csv` function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) can be used to load a single CSV file for our datset:

In [ ]:
import pandas as pd

In [ ]:
%%time

df = pd.read_csv(files[0])
df

The goal of this exercise is to use `dask.delayed` to create a new `read_csv_parallel` function which reads in *all* the files in the dataset in parallel using `dask.delayed`.

In [ ]:
# Your solution here

In [ ]:
%load solutions/delayed-3.py

# Additional Resources

- [Delayed documentation](https://docs.dask.org/en/latest/delayed.html)
- [Delayed screencast](https://www.youtube.com/watch?v=SHqFmynRxVU)
- [Delayed API](https://docs.dask.org/en/latest/delayed-api.html)
- [Delayed examples](https://examples.dask.org/delayed.html)
- [Delayed best practices](https://docs.dask.org/en/latest/delayed-best-practices.html)

# Next steps

Next, we'll move onto discussing [Dask DataFrames](2-dataframe.ipynb)